Trong hướng dẫn này, bạn sẽ học cách xây dựng và tối ưu hóa mô hình bằng **gradient boosting**.  

Phương pháp này thống trị nhiều cuộc thi Kaggle và đạt được kết quả tiên tiến trên nhiều tập dữ liệu khác nhau.

# Giới thiệu (# Introduction #)

Trong phần lớn khóa học này, bạn đã thực hiện dự đoán bằng phương pháp *random forest*, phương pháp này cho kết quả tốt hơn so với một cây quyết định đơn lẻ bằng cách lấy trung bình dự đoán của nhiều cây quyết định.

Chúng ta gọi *random forest* là một phương pháp **ensemble method** (*phương pháp tổng hợp*).  

Theo định nghĩa, **ensemble methods** kết hợp dự đoán của nhiều mô hình khác nhau (ví dụ: nhiều cây quyết định trong *random forests*).  

Tiếp theo, chúng ta sẽ tìm hiểu về một phương pháp *ensemble* khác, đó là **gradient boosting**.

# Gradient Boosting

**Gradient boosting** là một phương pháp hoạt động theo chu kỳ để bổ sung dần dần các mô hình vào một tập hợp (*ensemble*).  

Quá trình này bắt đầu bằng cách khởi tạo *ensemble* với một mô hình đơn lẻ, có thể cho ra dự đoán còn rất đơn giản.  
(Dù dự đoán ban đầu có thể rất không chính xác, nhưng các mô hình được thêm vào sau sẽ giúp sửa lỗi đó.)

Sau đó, quá trình huấn luyện diễn ra theo các bước sau:
- Đầu tiên, chúng ta sử dụng *ensemble* hiện tại để tạo dự đoán cho từng quan sát trong tập dữ liệu.  
  Để đưa ra dự đoán, chúng ta cộng các dự đoán từ tất cả các mô hình trong *ensemble*.  
- Các dự đoán này sau đó được sử dụng để tính toán *hàm mất mát* (*loss function*), ví dụ như [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) (*MSE*).  
- Tiếp theo, chúng ta sử dụng *hàm mất mát* để huấn luyện một mô hình mới và thêm vào *ensemble*.  
  Cụ thể, chúng ta tìm các tham số mô hình sao cho việc thêm mô hình này vào tập hợp sẽ giúp giảm *hàm mất mát*.  
  (*Lưu ý: "Gradient" trong "gradient boosting" đề cập đến việc sử dụng [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) trên *hàm mất mát* để tối ưu hóa tham số của mô hình mới.*)  
- Cuối cùng, mô hình mới được thêm vào *ensemble*, và ...
- ... lặp lại quy trình trên!

![tut6_boosting](https://storage.googleapis.com/kaggle-media/learn/images/MvCGENh.png)

# Ví dụ (# Example #)

Chúng ta bắt đầu bằng cách tải dữ liệu huấn luyện và kiểm định vào các biến:  
- `X_train`, `X_valid` (tập đặc trưng huấn luyện và kiểm định)  
- `y_train`, `y_valid` (tập mục tiêu huấn luyện và kiểm định)

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

Trong ví dụ này, bạn sẽ làm việc với thư viện **XGBoost**.  

**XGBoost** là viết tắt của **Extreme Gradient Boosting**, đây là một phiên bản cải tiến của *gradient boosting* với nhiều tính năng bổ sung, tập trung vào hiệu suất và tốc độ.  
(*Scikit-learn cũng có một phiên bản *gradient boosting*, nhưng XGBoost có một số ưu điểm kỹ thuật vượt trội.*)

Trong ô mã tiếp theo, chúng ta sẽ nhập API của *scikit-learn* cho XGBoost:  
[`xgboost.XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn).  

Điều này cho phép chúng ta xây dựng và huấn luyện mô hình theo cách tương tự như trong *scikit-learn*.  

Như bạn sẽ thấy trong đầu ra, lớp `XGBRegressor` có rất nhiều tham số có thể điều chỉnh — bạn sẽ sớm tìm hiểu về chúng!

In [49]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Chúng ta cũng thực hiện dự đoán và đánh giá mô hình.

In [50]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 235193.68586616346


# Điều chỉnh tham số (# Parameter Tuning #)

XGBoost có một số tham số có thể ảnh hưởng đáng kể đến độ chính xác và tốc độ huấn luyện.  

Những tham số quan trọng đầu tiên mà bạn cần hiểu bao gồm:

### `n_estimators`
Tham số `n_estimators` xác định số lần thực hiện chu kỳ huấn luyện được mô tả ở trên.  

Nó cũng chính là số lượng mô hình được đưa vào *ensemble*.

- Giá trị _quá thấp_ gây ra _underfitting_ (*khớp dưới*), dẫn đến dự đoán không chính xác trên cả dữ liệu huấn luyện và kiểm định.  
- Giá trị _quá cao_ gây ra _overfitting_ (*khớp quá mức*), khiến mô hình dự đoán chính xác trên tập huấn luyện nhưng lại kém chính xác trên tập kiểm định (_mà đây mới là điều chúng ta quan tâm_).  

Giá trị `n_estimators` thường nằm trong khoảng 100-1000, nhưng điều này phụ thuộc nhiều vào tham số `learning_rate` được đề cập bên dưới.

Dưới đây là đoạn mã để thiết lập số lượng mô hình trong *ensemble*:

In [51]:

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### `early_stopping_rounds`

Tham số `early_stopping_rounds` cung cấp một cách để tự động tìm giá trị tối ưu cho `n_estimators`.  

*Early stopping* (*dừng sớm*) khiến mô hình dừng huấn luyện khi điểm số kiểm định không còn cải thiện, ngay cả khi chưa đạt đến giới hạn tối đa của `n_estimators`.  

Cách tiếp cận thông minh là đặt một giá trị `n_estimators` lớn, sau đó sử dụng `early_stopping_rounds` để tìm thời điểm tối ưu để dừng lặp.

Vì yếu tố ngẫu nhiên có thể khiến điểm số kiểm định không cải thiện trong một vòng nhất định, bạn cần xác định số vòng liên tiếp có thể giảm trước khi dừng.  

Việc đặt `early_stopping_rounds=5` là một lựa chọn hợp lý. Trong trường hợp này, mô hình sẽ dừng sau 5 vòng liên tiếp mà điểm số kiểm định không cải thiện.

Khi sử dụng `early_stopping_rounds`, bạn cũng cần dành một phần dữ liệu để tính điểm kiểm định - điều này được thực hiện bằng cách thiết lập tham số `eval_set`.

Chúng ta có thể sửa đổi ví dụ trước để bao gồm *early stopping* như sau:

In [52]:
my_model = XGBRegressor(n_estimators=500, 
                        eval_metric="mae",  # Đặt eval_metric trong constructor
                        early_stopping_rounds=5)  # Đặt early_stopping_rounds trong constructor

my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)  # Không cần đặt early_stopping_rounds ở đây nữa

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Nếu sau này bạn muốn huấn luyện mô hình với toàn bộ dữ liệu, hãy đặt `n_estimators` bằng giá trị tối ưu mà bạn đã tìm thấy khi chạy với *early stopping*.

### `learning_rate`

Thay vì lấy dự đoán bằng cách cộng trực tiếp dự đoán từ từng mô hình thành phần, chúng ta có thể nhân các dự đoán đó với một số nhỏ (được gọi là **learning rate**) trước khi cộng vào.  

Điều này có nghĩa là mỗi cây (*tree*) được thêm vào *ensemble* sẽ có ảnh hưởng ít hơn. Do đó, chúng ta có thể đặt giá trị `n_estimators` cao hơn mà không bị *overfitting*.  

Nếu chúng ta sử dụng *early stopping*, số lượng cây phù hợp sẽ được xác định tự động.

Nhìn chung, một *learning rate* nhỏ kết hợp với một số lượng lớn *estimators* sẽ tạo ra mô hình XGBoost chính xác hơn. Tuy nhiên, mô hình cũng sẽ mất nhiều thời gian huấn luyện hơn do số vòng lặp (*iterations*) tăng lên.  

Mặc định, XGBoost đặt `learning_rate=0.1`.

Chỉnh sửa ví dụ trước để thay đổi *learning rate* sẽ cho ra đoạn mã sau:

In [53]:
my_model = XGBRegressor(n_estimators=1000, 
                        learning_rate=0.05, 
                        early_stopping_rounds=5)  # ✅ Đặt trong constructor

my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)  # Không cần đặt lại early_stopping_rounds ở đây

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### `n_jobs`

Đối với các tập dữ liệu lớn, khi thời gian chạy là một yếu tố quan trọng, bạn có thể sử dụng tính toán song song để huấn luyện mô hình nhanh hơn.  

Thông thường, tham số `n_jobs` được đặt bằng số lõi (*cores*) trên máy tính của bạn.  

Đối với các tập dữ liệu nhỏ, điều này không mang lại lợi ích đáng kể.

Mô hình thu được sẽ không chính xác hơn, vì vậy việc tối ưu hóa thời gian huấn luyện thường chỉ mang tính chất phụ. Tuy nhiên, nó hữu ích trong các tập dữ liệu lớn, nơi mà thời gian huấn luyện có thể rất lâu trong lệnh `fit`.

Dưới đây là phiên bản đã chỉnh sửa của ví dụ:

In [54]:
my_model = XGBRegressor(n_estimators=1000, 
                        learning_rate=0.05, 
                        n_jobs=4,
                        early_stopping_rounds=5)  # ✅ Đặt trong constructor

my_model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)  # Không cần đặt early_stopping_rounds ở đây nữa

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

# Kết luận (# Conclusion #)

[XGBoost](https://xgboost.readthedocs.io/en/latest/) là một thư viện hàng đầu để làm việc với dữ liệu dạng bảng tiêu chuẩn (*tabular data*)  
(loại dữ liệu được lưu trong *Pandas DataFrame*, trái ngược với các dạng dữ liệu khác như hình ảnh và video).  

Với việc tinh chỉnh tham số một cách cẩn thận, bạn có thể huấn luyện các mô hình có độ chính xác rất cao.

# Đến lượt bạn! (# Your Turn #)

Hãy huấn luyện mô hình XGBoost của riêng bạn trong **[bài tập tiếp theo](https://www.kaggle.com/kernels/fork/3370271)**!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*